# 🔢 Notebook 3: Word Embeddings - Rappresentare le Parole come Numeri

**Trasformiamo le parole in vettori!** 🧮

## 🎯 Obiettivi di questo notebook:
- Comprendere perché l'AI ha bisogno di numeri
- Implementare Bag of Words (BoW)
- Calcolare TF-IDF da zero
- Esplorare Word2Vec pre-addestrato
- Visualizzare embeddings con t-SNE
- Confrontare diversi metodi di rappresentazione
- Applicare embeddings a problemi reali

## ⏱️ Tempo stimato: 75-90 minuti

## 📋 Prerequisiti: 
- Notebook 1 e 2 completati
- Concetti base di algebra lineare (vettori, distanze)
- Familiarità con la pre-elaborazione del testo

In [ ]:
# Installiamo le librerie necessarie
!pip install gensim scikit-learn matplotlib seaborn pandas numpy nltk spacy wordcloud plotly
!pip install umap-learn  # Per visualizzazioni alternative

## 🔧 Setup e Installazione

Installiamo le librerie necessarie per lavorare con gli embeddings:

In [1]:

# Importiamo le librerie
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter, defaultdict
import re
import math

# Librerie per NLP
import nltk
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Librerie per embeddings
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import gensim
from gensim.models import Word2Vec
from gensim.models.keyedvectors import KeyedVectors

# Visualizzazioni
from wordcloud import WordCloud
import plotly.express as px
import plotly.graph_objects as go

# Download dati NLTK
nltk.download('punkt')
nltk.download('stopwords')

# Configurazione grafica
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

print("✅ Setup completato! Pronti per esplorare i Word Embeddings.")

ModuleNotFoundError: No module named 'spacy'

## 🤔 Perché l'AI Ha Bisogno di Numeri?

I computer non capiscono le parole, ma solo numeri. Vediamo il problema:

In [ ]:
# Il problema della rappresentazione
print("🤖 IL PROBLEMA: Come fa un computer a capire le parole?\n")

# Esempio di parole
parole = ["gatto", "cane", "automobile", "felice", "triste"]

print("📝 PAROLE UMANE:")
for i, parola in enumerate(parole):
    print(f"   {i+1}. {parola}")

print("\n🔢 RAPPRESENTAZIONE COMPUTER (ASCII):")
for i, parola in enumerate(parole):
    ascii_vals = [ord(char) for char in parola]
    print(f"   {i+1}. {parola} → {ascii_vals}")

print("\n❌ PROBLEMI CON ASCII:")
print("   • Lunghezze diverse")
print("   • Nessuna relazione semantica")
print("   • 'gatto' e 'cane' sembrano completamente diversi")
print("   • Non cattura il significato")

# Visualizziamo il problema
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Grafico 1: Lunghezze parole
lunghezze = [len(parola) for parola in parole]
ax1.bar(parole, lunghezze, color='lightcoral', alpha=0.7)
ax1.set_title('Lunghezze Parole (Caratteri)')
ax1.set_ylabel('Numero Caratteri')
ax1.tick_params(axis='x', rotation=45)

# Grafico 2: Somma valori ASCII
ascii_sums = [sum(ord(char) for char in parola) for parola in parole]
ax2.bar(parole, ascii_sums, color='lightblue', alpha=0.7)
ax2.set_title('Somma Valori ASCII')
ax2.set_ylabel('Somma ASCII')
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print("\n💡 SOLUZIONE: Word Embeddings!")
print("   • Rappresentano parole come vettori di numeri")
print("   • Catturano relazioni semantiche")
print("   • Parole simili hanno vettori simili")
print("   • Dimensione fissa per tutte le parole")

# Esempio concettuale di embeddings
print("\n🎯 ESEMPIO CONCETTUALE (2D):")
embeddings_esempio = {
    'gatto': [0.8, 0.9],    # Animale domestico
    'cane': [0.9, 0.8],     # Animale domestico (simile a gatto)
    'automobile': [-0.5, 0.1],  # Oggetto meccanico
    'felice': [0.2, -0.8],  # Emozione positiva
    'triste': [0.1, 0.7]    # Emozione negativa
}

# Visualizziamo gli embeddings concettuali
fig, ax = plt.subplots(figsize=(10, 8))

for parola, (x, y) in embeddings_esempio.items():
    ax.scatter(x, y, s=200, alpha=0.7)
    ax.annotate(parola, (x, y), xytext=(5, 5), textcoords='offset points',
                fontsize=12, fontweight='bold')

ax.set_xlabel('Dimensione 1 (es. "Animale vs Oggetto")')
ax.set_ylabel('Dimensione 2 (es. "Domestico vs Selvaggio")')
ax.set_title('Embeddings Concettuali 2D')
ax.grid(True, alpha=0.3)
ax.axhline(y=0, color='k', linestyle='-', alpha=0.3)
ax.axvline(x=0, color='k', linestyle='-', alpha=0.3)

plt.tight_layout()
plt.show()

print("👀 OSSERVAZIONI:")
print("   • 'gatto' e 'cane' sono vicini (entrambi animali domestici)")
print("   • 'automobile' è lontano (oggetto meccanico)")
print("   • 'felice' e 'triste' sono in zone diverse (emozioni opposte)")

## 🎒 Bag of Words (BoW) - Il Metodo Base

Iniziamo con il metodo più semplice: **Bag of Words**

In [ ]:
# Implementiamo Bag of Words da zero
class BagOfWords:
    def __init__(self):
        self.vocabolario = {}
        self.vocabolario_inverso = {}
        self.dimensione_vocab = 0
    
    def costruisci_vocabolario(self, documenti):
        """Costruisce il vocabolario da una lista di documenti"""
        tutte_parole = set()
        
        for doc in documenti:
            parole = doc.lower().split()
            tutte_parole.update(parole)
        
        # Crea mappatura parola -> indice
        for i, parola in enumerate(sorted(tutte_parole)):
            self.vocabolario[parola] = i
            self.vocabolario_inverso[i] = parola
        
        self.dimensione_vocab = len(self.vocabolario)
        print(f"📚 Vocabolario costruito: {self.dimensione_vocab} parole uniche")
    
    def documento_to_vettore(self, documento):
        """Converte un documento in vettore BoW"""
        vettore = np.zeros(self.dimensione_vocab)
        parole = documento.lower().split()
        
        for parola in parole:
            if parola in self.vocabolario:
                indice = self.vocabolario[parola]
                vettore[indice] += 1
        
        return vettore
    
    def trasforma_documenti(self, documenti):
        """Trasforma una lista di documenti in matrice BoW"""
        matrice = []
        for doc in documenti:
            vettore = self.documento_to_vettore(doc)
            matrice.append(vettore)
        return np.array(matrice)

# Test con documenti di esempio
print("🎒 BAG OF WORDS - IMPLEMENTAZIONE DA ZERO\n")

documenti_esempio = [
    "il gatto dorme sul divano",
    "il cane corre nel parco",
    "il gatto e il cane sono amici",
    "sul divano c'è un gatto nero",
    "nel parco corrono molti cani"
]

print("📝 DOCUMENTI DI ESEMPIO:")
for i, doc in enumerate(documenti_esempio, 1):
    print(f"   {i}. {doc}")

# Costruiamo il modello BoW
bow = BagOfWords()
bow.costruisci_vocabolario(documenti_esempio)

print(f"\n📚 VOCABOLARIO ({bow.dimensione_vocab} parole):")
print(list(bow.vocabolario.keys()))

# Trasformiamo i documenti
matrice_bow = bow.trasforma_documenti(documenti_esempio)

print(f"\n🔢 MATRICE BAG OF WORDS ({matrice_bow.shape[0]} documenti x {matrice_bow.shape[1]} parole):")

# Creiamo un DataFrame per visualizzare meglio
df_bow = pd.DataFrame(matrice_bow, 
                      columns=list(bow.vocabolario.keys()),
                      index=[f"Doc {i+1}" for i in range(len(documenti_esempio))])

print(df_bow)

# Visualizzazione della matrice
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Heatmap della matrice BoW
sns.heatmap(df_bow, annot=True, cmap='Blues', ax=ax1, cbar_kws={'label': 'Frequenza'})
ax1.set_title('Matrice Bag of Words')
ax1.set_xlabel('Parole')
ax1.set_ylabel('Documenti')

# Frequenze totali delle parole
freq_parole = matrice_bow.sum(axis=0)
parole_ordinate = sorted(bow.vocabolario.keys(), key=lambda x: freq_parole[bow.vocabolario[x]], reverse=True)
freq_ordinate = [freq_parole[bow.vocabolario[p]] for p in parole_ordinate[:10]]

ax2.bar(parole_ordinate[:10], freq_ordinate, color='lightblue', alpha=0.7)
ax2.set_title('Top 10 Parole Più Frequenti')
ax2.set_xlabel('Parole')
ax2.set_ylabel('Frequenza Totale')
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print("\n💡 CARATTERISTICHE BAG OF WORDS:")
print("✅ Vantaggi:")
print("   • Semplice da implementare")
print("   • Interpretabile")
print("   • Veloce")
print("   • Buono per classificazione base")

print("\n❌ Svantaggi:")
print("   • Ignora l'ordine delle parole")
print("   • Vettori molto sparsi (molti zeri)")
print("   • Dimensionalità alta")
print("   • Non cattura relazioni semantiche")

# Esempio di similarità
print("\n🔍 ESEMPIO SIMILARITÀ:")
similarita = cosine_similarity(matrice_bow)
print(f"Similarità Doc 1 vs Doc 3: {similarita[0][2]:.3f}")
print(f"Similarità Doc 1 vs Doc 4: {similarita[0][3]:.3f}")
print("(Doc 1 e 4 parlano entrambi di gatti sul divano)")

## 📊 TF-IDF - Pesare l'Importanza delle Parole

**TF-IDF** (Term Frequency - Inverse Document Frequency) migliora BoW pesando l'importanza delle parole:

In [ ]:
# Implementiamo TF-IDF da zero
class TFIDF:
    def __init__(self):
        self.vocabolario = {}
        self.idf_scores = {}
        self.dimensione_vocab = 0
    
    def calcola_tf(self, documento):
        """Calcola Term Frequency per un documento"""
        parole = documento.lower().split()
        tf = Counter(parole)
        
        # Normalizza per la lunghezza del documento
        lunghezza_doc = len(parole)
        for parola in tf:
            tf[parola] = tf[parola] / lunghezza_doc
        
        return tf
    
    def calcola_idf(self, documenti):
        """Calcola Inverse Document Frequency"""
        n_documenti = len(documenti)
        
        # Conta in quanti documenti appare ogni parola
        df = defaultdict(int)  # document frequency
        
        for doc in documenti:
            parole_uniche = set(doc.lower().split())
            for parola in parole_uniche:
                df[parola] += 1
        
        # Calcola IDF
        for parola, freq_doc in df.items():
            self.idf_scores[parola] = math.log(n_documenti / freq_doc)
        
        # Costruisci vocabolario
        for i, parola in enumerate(sorted(df.keys())):
            self.vocabolario[parola] = i
        
        self.dimensione_vocab = len(self.vocabolario)
    
    def documento_to_tfidf(self, documento):
        """Converte documento in vettore TF-IDF"""
        tf = self.calcola_tf(documento)
        vettore = np.zeros(self.dimensione_vocab)
        
        for parola, tf_score in tf.items():
            if parola in self.vocabolario:
                indice = self.vocabolario[parola]
                idf_score = self.idf_scores[parola]
                vettore[indice] = tf_score * idf_score
        
        return vettore
    
    def fit_transform(self, documenti):
        """Addestra e trasforma i documenti"""
        self.calcola_idf(documenti)
        
        matrice = []
        for doc in documenti:
            vettore = self.documento_to_tfidf(doc)
            matrice.append(vettore)
        
        return np.array(matrice)

# Test TF-IDF
print("📊 TF-IDF - IMPLEMENTAZIONE DA ZERO\n")

# Usiamo gli stessi documenti
tfidf = TFIDF()
matrice_tfidf = tfidf.fit_transform(documenti_esempio)

print(f"📚 Vocabolario TF-IDF: {tfidf.dimensione_vocab} parole")

# Mostriamo i punteggi IDF
print("\n🔢 PUNTEGGI IDF (Inverse Document Frequency):")
idf_ordinati = sorted(tfidf.idf_scores.items(), key=lambda x: x[1], reverse=True)
for parola, idf in idf_ordinati:
    print(f"   '{parola}': {idf:.3f}")

print("\n💡 INTERPRETAZIONE IDF:")
print("   • Punteggio ALTO = parola rara (appare in pochi documenti)")
print("   • Punteggio BASSO = parola comune (appare in molti documenti)")
print("   • 'il' ha IDF basso perché appare ovunque")
print("   • 'nero' ha IDF alto perché appare solo in un documento")

# Confronto BoW vs TF-IDF
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# DataFrame per TF-IDF
df_tfidf = pd.DataFrame(matrice_tfidf, 
                        columns=list(tfidf.vocabolario.keys()),
                        index=[f"Doc {i+1}" for i in range(len(documenti_esempio))])

# Heatmap BoW
sns.heatmap(df_bow, annot=True, cmap='Blues', ax=axes[0,0], 
            cbar_kws={'label': 'Frequenza'})
axes[0,0].set_title('Bag of Words')

# Heatmap TF-IDF
sns.heatmap(df_tfidf, annot=True, fmt='.2f', cmap='Reds', ax=axes[0,1],
            cbar_kws={'label': 'TF-IDF Score'})
axes[0,1].set_title('TF-IDF')

# Confronto punteggi per una parola specifica
parola_esempio = 'gatto'
if parola_esempio in bow.vocabolario and parola_esempio in tfidf.vocabolario:
    idx_bow = bow.vocabolario[parola_esempio]
    idx_tfidf = tfidf.vocabolario[parola_esempio]
    
    bow_scores = matrice_bow[:, idx_bow]
    tfidf_scores = matrice_tfidf[:, idx_tfidf]
    
    x = np.arange(len(documenti_esempio))
    width = 0.35
    
    axes[1,0].bar(x - width/2, bow_scores, width, label='BoW', alpha=0.7, color='blue')
    axes[1,0].bar(x + width/2, tfidf_scores, width, label='TF-IDF', alpha=0.7, color='red')
    axes[1,0].set_title(f'Confronto Punteggi: "{parola_esempio}"')
    axes[1,0].set_xlabel('Documenti')
    axes[1,0].set_ylabel('Punteggio')
    axes[1,0].set_xticks(x)
    axes[1,0].set_xticklabels([f'Doc {i+1}' for i in range(len(documenti_esempio))])
    axes[1,0].legend()

# Distribuzione punteggi IDF
idf_values = list(tfidf.idf_scores.values())
axes[1,1].hist(idf_values, bins=10, alpha=0.7, color='green', edgecolor='black')
axes[1,1].set_title('Distribuzione Punteggi IDF')
axes[1,1].set_xlabel('Punteggio IDF')
axes[1,1].set_ylabel('Frequenza')

plt.tight_layout()
plt.show()

# Confronto similarità
print("\n🔍 CONFRONTO SIMILARITÀ:")
sim_bow = cosine_similarity(matrice_bow)
sim_tfidf = cosine_similarity(matrice_tfidf)

print("\nSimilarità Doc 1 vs Doc 4 (entrambi parlano di gatti):")
print(f"   BoW: {sim_bow[0][3]:.3f}")
print(f"   TF-IDF: {sim_tfidf[0][3]:.3f}")

print("\nSimilarità Doc 2 vs Doc 5 (entrambi parlano di cani nel parco):")
print(f"   BoW: {sim_bow[1][4]:.3f}")
print(f"   TF-IDF: {sim_tfidf[1][4]:.3f}")

print("\n💡 VANTAGGI TF-IDF:")
print("✅ Riduce l'importanza di parole comuni")
print("✅ Evidenzia parole distintive")
print("✅ Migliore per ricerca e classificazione")
print("✅ Normalizza per lunghezza documento")

print("\n❌ LIMITI TF-IDF:")
print("❌ Ancora ignora l'ordine delle parole")
print("❌ Non cattura sinonimi")
print("❌ Vettori ancora sparsi")
print("❌ Non comprende il contesto")

## 🧠 Word2Vec - Embeddings Intelligenti

**Word2Vec** rappresenta un salto qualitativo: impara rappresentazioni dense che catturano relazioni semantiche:

In [ ]:
# Scarichiamo un modello Word2Vec pre-addestrato
print("🧠 WORD2VEC - EMBEDDINGS INTELLIGENTI\n")

# Per questo esempio, creeremo un piccolo corpus e addestriamo Word2Vec
# In pratica, usereste modelli pre-addestrati su grandi corpus

# Corpus più grande per addestrare Word2Vec
corpus_esteso = [
    "il gatto dorme sul divano comodo",
    "il cane corre veloce nel parco verde",
    "i gatti amano dormire al sole caldo",
    "i cani giocano insieme nel parco",
    "il divano è molto comodo per dormire",
    "nel parco ci sono molti alberi verdi",
    "gli animali domestici sono fedeli compagni",
    "il sole splende caldo in giardino",
    "dormire è importante per la salute",
    "correre fa bene alla salute fisica",
    "i compagni fedeli sono preziosi",
    "il giardino verde è pieno di fiori",
    "gli alberi danno ombra fresca",
    "i fiori colorati profumano dolcemente",
    "la salute fisica è molto importante",
    "gli animali selvatici vivono liberi",
    "la natura è bella e selvaggia",
    "i colori della natura sono vivaci",
    "l'ombra fresca protegge dal caldo",
    "i profumi dolci attirano le api"
]

print(f"📚 CORPUS ESTESO: {len(corpus_esteso)} frasi")

# Prepariamo i dati per Word2Vec
frasi_tokenizzate = []
for frase in corpus_esteso:
    tokens = frase.lower().split()
    frasi_tokenizzate.append(tokens)

print(f"🔤 FRASI TOKENIZZATE: {len(frasi_tokenizzate)} frasi")
print("Esempio:", frasi_tokenizzate[0])

# Addestriamo Word2Vec
print("\n🏋️ ADDESTRAMENTO WORD2VEC...")
model_w2v = Word2Vec(
    sentences=frasi_tokenizzate,
    vector_size=50,      # Dimensione degli embeddings
    window=5,            # Finestra di contesto
    min_count=1,         # Frequenza minima parole
    workers=4,           # Thread paralleli
    sg=0,                # 0=CBOW, 1=Skip-gram
    epochs=100           # Numero di epoche
)

print("✅ Addestramento completato!")
print(f"📊 Vocabolario: {len(model_w2v.wv.key_to_index)} parole")
print(f"🔢 Dimensione embeddings: {model_w2v.wv.vector_size}")

# Esploriamo gli embeddings
print("\n🔍 ESPLORAZIONE EMBEDDINGS:")

# Parole nel vocabolario
vocabolario_w2v = list(model_w2v.wv.key_to_index.keys())
print(f"Parole nel vocabolario: {vocabolario_w2v[:10]}...")

# Embedding di una parola specifica
parola_test = 'gatto'
if parola_test in model_w2v.wv:
    embedding_gatto = model_w2v.wv[parola_test]
    print(f"\n🐱 Embedding di '{parola_test}':")
    print(f"   Dimensioni: {embedding_gatto.shape}")
    print(f"   Primi 10 valori: {embedding_gatto[:10]}")

# Parole più simili
print(f"\n🔗 PAROLE PIÙ SIMILI A '{parola_test}':")
try:
    simili = model_w2v.wv.most_similar(parola_test, topn=5)
    for parola, similarita in simili:
        print(f"   '{parola}': {similarita:.3f}")
except KeyError:
    print(f"   '{parola_test}' non trovato nel vocabolario")

# Test analogie (se possibile con il nostro piccolo corpus)
print("\n🧮 TEST ANALOGIE:")
try:
    # gatto : dorme = cane : ?
    analogia = model_w2v.wv.most_similar(
        positive=['cane', 'dorme'], 
        negative=['gatto'], 
        topn=3
    )
    print("gatto : dorme = cane : ?")
    for parola, score in analogia:
        print(f"   {parola} ({score:.3f})")
except:
    print("   Analogie non disponibili con questo corpus piccolo")

# Visualizzazione degli embeddings
print("\n📊 VISUALIZZAZIONE EMBEDDINGS (PCA 2D):")

# Selezioniamo alcune parole interessanti
parole_interesse = ['gatto', 'cane', 'divano', 'parco', 'dormire', 'correre', 
                   'sole', 'alberi', 'salute', 'natura']
parole_disponibili = [p for p in parole_interesse if p in model_w2v.wv]

if len(parole_disponibili) > 3:
    # Ottieni embeddings
    embeddings_viz = np.array([model_w2v.wv[parola] for parola in parole_disponibili])
    
    # Riduci dimensionalità con PCA
    pca = PCA(n_components=2)
    embeddings_2d = pca.fit_transform(embeddings_viz)
    
    # Visualizza
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Scatter plot
    scatter = ax1.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], 
                         s=100, alpha=0.7, c=range(len(parole_disponibili)), 
                         cmap='tab10')
    
    for i, parola in enumerate(parole_disponibili):
        ax1.annotate(parola, (embeddings_2d[i, 0], embeddings_2d[i, 1]),
                    xytext=(5, 5), textcoords='offset points',
                    fontsize=10, fontweight='bold')
    
    ax1.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} varianza)')
    ax1.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} varianza)')
    ax1.set_title('Word2Vec Embeddings (PCA 2D)')
    ax1.grid(True, alpha=0.3)
    
    # Matrice di similarità
    n_parole = len(parole_disponibili)
    matrice_sim = np.zeros((n_parole, n_parole))
    
    for i, parola1 in enumerate(parole_disponibili):
        for j, parola2 in enumerate(parole_disponibili):
            if i != j:
                sim = model_w2v.wv.similarity(parola1, parola2)
                matrice_sim[i, j] = sim
            else:
                matrice_sim[i, j] = 1.0
    
    sns.heatmap(matrice_sim, 
                xticklabels=parole_disponibili,
                yticklabels=parole_disponibili,
                annot=True, fmt='.2f', cmap='coolwarm',
                center=0, ax=ax2)
    ax2.set_title('Matrice Similarità Word2Vec')
    
    plt.tight_layout()
    plt.show()
    
    print("👀 OSSERVAZIONI:")
    print("   • Parole semanticamente simili sono vicine nello spazio")
    print("   • Gli embeddings catturano relazioni non evidenti in BoW/TF-IDF")
    print("   • La dimensionalità è molto più bassa ma più informativa")

print("\n💡 VANTAGGI WORD2VEC:")
print("✅ Cattura relazioni semantiche")
print("✅ Embeddings densi (pochi zeri)")
print("✅ Dimensionalità controllabile")
print("✅ Supporta analogie")
print("✅ Transfer learning possibile")

print("\n❌ LIMITI WORD2VEC:")
print("❌ Una rappresentazione per parola (no polisemia)")
print("❌ Non gestisce parole fuori vocabolario")
print("❌ Richiede grandi corpus per qualità")
print("❌ Non considera contesto della frase")

## 🔍 Confronto Completo dei Metodi

Confrontiamo tutti i metodi su un task pratico:

In [ ]:
# Confronto completo dei metodi
print("🔍 CONFRONTO COMPLETO: BoW vs TF-IDF vs Word2Vec\n")

# Task: Trovare documenti simili
query = "gatto che dorme"
print(f"🎯 QUERY: '{query}'")
print(f"📚 DOCUMENTI DA CERCARE:")
for i, doc in enumerate(documenti_esempio, 1):
    print(f"   {i}. {doc}")

# 1. BoW
print("\n1️⃣ RICERCA CON BAG OF WORDS:")
query_bow = bow.documento_to_vettore(query)
sim_bow_query = cosine_similarity([query_bow], matrice_bow)[0]

risultati_bow = [(i+1, sim, doc) for i, (sim, doc) in enumerate(zip(sim_bow_query, documenti_esempio))]
risultati_bow.sort(key=lambda x: x[1], reverse=True)

for rank, (doc_id, similarita, doc) in enumerate(risultati_bow[:3], 1):
    print(f"   {rank}. Doc {doc_id} (sim: {similarita:.3f}): {doc}")

# 2. TF-IDF
print("\n2️⃣ RICERCA CON TF-IDF:")
query_tfidf = tfidf.documento_to_tfidf(query)
sim_tfidf_query = cosine_similarity([query_tfidf], matrice_tfidf)[0]

risultati_tfidf = [(i+1, sim, doc) for i, (sim, doc) in enumerate(zip(sim_tfidf_query, documenti_esempio))]
risultati_tfidf.sort(key=lambda x: x[1], reverse=True)

for rank, (doc_id, similarita, doc) in enumerate(risultati_tfidf[:3], 1):
    print(f"   {rank}. Doc {doc_id} (sim: {similarita:.3f}): {doc}")

# 3. Word2Vec (media degli embeddings)
print("\n3️⃣ RICERCA CON WORD2VEC:")

def documento_to_w2v(documento, model):
    """Converte documento in embedding medio"""
    parole = documento.lower().split()
    embeddings = []
    
    for parola in parole:
        if parola in model.wv:
            embeddings.append(model.wv[parola])
    
    if embeddings:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(model.wv.vector_size)

# Converti query e documenti
query_w2v = documento_to_w2v(query, model_w2v)
docs_w2v = [documento_to_w2v(doc, model_w2v) for doc in documenti_esempio]

# Calcola similarità
sim_w2v_query = cosine_similarity([query_w2v], docs_w2v)[0]

risultati_w2v = [(i+1, sim, doc) for i, (sim, doc) in enumerate(zip(sim_w2v_query, documenti_esempio))]
risultati_w2v.sort(key=lambda x: x[1], reverse=True)

for rank, (doc_id, similarita, doc) in enumerate(risultati_w2v[:3], 1):
    print(f"   {rank}. Doc {doc_id} (sim: {similarita:.3f}): {doc}")

# Visualizzazione comparativa
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

metodi = ['BoW', 'TF-IDF', 'Word2Vec']
risultati_tutti = [risultati_bow, risultati_tfidf, risultati_w2v]

for i, (metodo, risultati) in enumerate(zip(metodi, risultati_tutti)):
    doc_ids = [r[0] for r in risultati]
    similarita = [r[1] for r in risultati]
    
    bars = axes[i].bar(doc_ids, similarita, alpha=0.7, 
                      color=['gold' if j == 0 else 'lightblue' for j in range(len(doc_ids))])
    
    axes[i].set_title(f'Similarità con Query - {metodo}')
    axes[i].set_xlabel('Documento ID')
    axes[i].set_ylabel('Similarità Coseno')
    axes[i].set_ylim(0, max(similarita) * 1.1 if max(similarita) > 0 else 1)
    
    # Evidenzia il migliore
    best_idx = np.argmax(similarita)
    bars[best_idx].set_color('gold')
    
    # Aggiungi valori sopra le barre
    for bar, sim in zip(bars, similarita):
        if sim > 0:
            axes[i].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                        f'{sim:.2f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

# Tabella riassuntiva
print("\n📊 TABELLA RIASSUNTIVA:")
df_confronto = pd.DataFrame({
    'Metodo': ['BoW', 'TF-IDF', 'Word2Vec'],
    'Miglior Match': [
        f"Doc {risultati_bow[0][0]} ({risultati_bow[0][1]:.3f})",
        f"Doc {risultati_tfidf[0][0]} ({risultati_tfidf[0][1]:.3f})",
        f"Doc {risultati_w2v[0][0]} ({risultati_w2v[0][1]:.3f})"
    ],
    'Documento': [
        risultati_bow[0][2],
        risultati_tfidf[0][2],
        risultati_w2v[0][2]
    ]
})

print(df_confronto.to_string(index=False))

print("\n💡 ANALISI RISULTATI:")
print("• BoW: Trova corrispondenze esatte di parole")
print("• TF-IDF: Bilancia frequenza e rarità")
print("• Word2Vec: Cattura similarità semantica")
print("• Ogni metodo ha i suoi punti di forza")

print("\n🎯 QUANDO USARE COSA:")
print("📊 BoW: Classificazione semplice, analisi frequenze")
print("📈 TF-IDF: Ricerca documenti, classificazione testi")
print("🧠 Word2Vec: Analisi semantica, raccomandazioni, AI avanzata")

## 🎨 Visualizzazione Avanzata con t-SNE

Usiamo **t-SNE** per visualizzare gli embeddings in modo più intuitivo:

In [ ]:
# Visualizzazione avanzata con t-SNE
print("🎨 VISUALIZZAZIONE AVANZATA CON t-SNE\n")

# Prepariamo i dati per t-SNE
if len(vocabolario_w2v) > 10:  # Assicuriamoci di avere abbastanza parole
    
    # Selezioniamo parole interessanti per la visualizzazione
    parole_viz = []
    categorie = {
        'Animali': ['gatto', 'cane', 'animali'],
        'Luoghi': ['divano', 'parco', 'giardino', 'casa'],
        'Azioni': ['dormire', 'correre', 'giocare'],
        'Natura': ['sole', 'alberi', 'fiori', 'natura'],
        'Qualità': ['comodo', 'verde', 'caldo', 'bello']
    }
    
    parole_selezionate = []
    etichette_categoria = []
    
    for categoria, parole in categorie.items():
        for parola in parole:
            if parola in model_w2v.wv:
                parole_selezionate.append(parola)
                etichette_categoria.append(categoria)
    
    if len(parole_selezionate) > 5:
        print(f"📊 Visualizzando {len(parole_selezionate)} parole in {len(set(etichette_categoria))} categorie")
        
        # Ottieni embeddings
        embeddings_tsne = np.array([model_w2v.wv[parola] for parola in parole_selezionate])
        
        # Applica t-SNE
        print("🔄 Applicando t-SNE...")
        tsne = TSNE(n_components=2, random_state=42, perplexity=min(5, len(parole_selezionate)-1))
        embeddings_tsne_2d = tsne.fit_transform(embeddings_tsne)
        
        # Crea DataFrame per visualizzazione
        df_viz = pd.DataFrame({
            'x': embeddings_tsne_2d[:, 0],
            'y': embeddings_tsne_2d[:, 1],
            'parola': parole_selezionate,
            'categoria': etichette_categoria
        })
        
        # Visualizzazione con matplotlib
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))
        
        # Plot 1: Colorato per categoria
        categorie_uniche = list(set(etichette_categoria))
        colori = plt.cm.Set3(np.linspace(0, 1, len(categorie_uniche)))
        
        for i, categoria in enumerate(categorie_uniche):
            mask = df_viz['categoria'] == categoria
            ax1.scatter(df_viz[mask]['x'], df_viz[mask]['y'], 
                       c=[colori[i]], label=categoria, s=100, alpha=0.7)
            
            # Aggiungi etichette
            for _, row in df_viz[mask].iterrows():
                ax1.annotate(row['parola'], (row['x'], row['y']),
                           xytext=(5, 5), textcoords='offset points',
                           fontsize=9, fontweight='bold')
        
        ax1.set_title('t-SNE Word2Vec Embeddings (per Categoria)')
        ax1.set_xlabel('t-SNE Dimensione 1')
        ax1.set_ylabel('t-SNE Dimensione 2')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # Plot 2: Heatmap distanze
        n_parole_viz = len(parole_selezionate)
        matrice_distanze = np.zeros((n_parole_viz, n_parole_viz))
        
        for i in range(n_parole_viz):
            for j in range(n_parole_viz):
                if i != j:
                    # Distanza euclidea nello spazio t-SNE
                    dist = np.linalg.norm(embeddings_tsne_2d[i] - embeddings_tsne_2d[j])
                    matrice_distanze[i, j] = dist
        
        sns.heatmap(matrice_distanze,
                   xticklabels=parole_selezionate,
                   yticklabels=parole_selezionate,
                   annot=True, fmt='.1f', cmap='viridis_r',
                   ax=ax2)
        ax2.set_title('Matrice Distanze t-SNE')
        
        plt.tight_layout()
        plt.show()
        
        # Analisi dei cluster
        print("\n🔍 ANALISI CLUSTER:")
        for categoria in categorie_uniche:
            parole_cat = df_viz[df_viz['categoria'] == categoria]['parola'].tolist()
            if len(parole_cat) > 1:
                print(f"\n📂 {categoria}:")
                for parola in parole_cat:
                    print(f"   • {parola}")
                
                # Calcola coesione del cluster
                embeddings_cat = [model_w2v.wv[p] for p in parole_cat]
                if len(embeddings_cat) > 1:
                    sim_interna = []
                    for i in range(len(embeddings_cat)):
                        for j in range(i+1, len(embeddings_cat)):
                            sim = cosine_similarity([embeddings_cat[i]], [embeddings_cat[j]])[0][0]
                            sim_interna.append(sim)
                    
                    coesione = np.mean(sim_interna)
                    print(f"   Coesione cluster: {coesione:.3f}")
        
        print("\n💡 INTERPRETAZIONE t-SNE:")
        print("• Parole vicine hanno significati correlati")
        print("• I cluster riflettono categorie semantiche")
        print("• t-SNE preserva le vicinanze locali")
        print("• Utile per esplorare il vocabolario")
        
    else:
        print("⚠️ Poche parole disponibili per t-SNE")
else:
    print("⚠️ Vocabolario troppo piccolo per visualizzazione avanzata")

print("\n🎯 APPLICAZIONI PRATICHE t-SNE:")
print("🔍 Esplorazione vocabolario")
print("🏷️ Identificazione cluster semantici")
print("🐛 Debug di modelli di embeddings")
print("📊 Visualizzazione per presentazioni")
print("🔬 Ricerca in linguistica computazionale")

## 🛠️ Applicazione Pratica: Sistema di Raccomandazione

Costruiamo un sistema di raccomandazione usando gli embeddings:

In [ ]:
# Sistema di raccomandazione con embeddings
class SistemaRaccomandazione:
    def __init__(self, metodo='word2vec'):
        self.metodo = metodo
        self.documenti = []
        self.embeddings = []
        self.titoli = []
        
    def aggiungi_documenti(self, documenti, titoli=None):
        """Aggiunge documenti al sistema"""
        self.documenti = documenti
        self.titoli = titoli if titoli else [f"Doc {i+1}" for i in range(len(documenti))]
        
        # Calcola embeddings in base al metodo
        if self.metodo == 'tfidf':
            vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
            self.embeddings = vectorizer.fit_transform(documenti).toarray()
            
        elif self.metodo == 'word2vec':
            self.embeddings = []
            for doc in documenti:
                embedding = documento_to_w2v(doc, model_w2v)
                self.embeddings.append(embedding)
            self.embeddings = np.array(self.embeddings)
    
    def raccomanda(self, query, n_raccomandazioni=3):
        """Raccomanda documenti simili alla query"""
        if self.metodo == 'tfidf':
            # Per TF-IDF dovremmo usare lo stesso vectorizer
            # Semplificazione: usiamo Word2Vec
            query_embedding = documento_to_w2v(query, model_w2v)
        else:
            query_embedding = documento_to_w2v(query, model_w2v)
        
        # Calcola similarità
        similarita = cosine_similarity([query_embedding], self.embeddings)[0]
        
        # Ordina per similarità
        indici_ordinati = np.argsort(similarita)[::-1]
        
        raccomandazioni = []
        for i in range(min(n_raccomandazioni, len(indici_ordinati))):
            idx = indici_ordinati[i]
            raccomandazioni.append({
                'titolo': self.titoli[idx],
                'documento': self.documenti[idx],
                'similarita': similarita[idx],
                'rank': i + 1
            })
        
        return raccomandazioni

# Test del sistema di raccomandazione
print("🛠️ SISTEMA DI RACCOMANDAZIONE CON EMBEDDINGS\n")

# Dataset di esempio (articoli di blog)
articoli = [
    "Come addestrare il tuo gatto a usare la lettiera. Consigli pratici per proprietari di gatti.",
    "I migliori parchi per cani nella tua città. Dove portare il tuo cane a giocare.",
    "Ricette salutari per una dieta equilibrata. Mangiare bene per stare in forma.",
    "Esercizi di yoga per rilassarsi dopo una giornata stressante. Trova la pace interiore.",
    "Giardinaggio urbano: come coltivare piante in appartamento. Verde in casa.",
    "Tecniche di meditazione per principianti. Inizia il tuo percorso di mindfulness.",
    "Cura degli animali domestici: consigli veterinari per gatti e cani.",
    "Alimentazione naturale per animali: cosa dare da mangiare ai tuoi pets.",
    "Creare un giardino zen in casa. Spazi di tranquillità domestici.",
    "Sport e benessere: l'importanza dell'attività fisica per la salute mentale."
]

titoli_articoli = [
    "Addestramento Gatti",
    "Parchi per Cani", 
    "Ricette Salutari",
    "Yoga e Relax",
    "Giardinaggio Urbano",
    "Meditazione Base",
    "Cura Animali",
    "Alimentazione Pets",
    "Giardino Zen",
    "Sport e Benessere"
]

print("📚 DATASET ARTICOLI:")
for i, (titolo, articolo) in enumerate(zip(titoli_articoli, articoli), 1):
    print(f"   {i}. {titolo}: {articolo[:50]}...")

# Inizializza sistema
sistema = SistemaRaccomandazione(metodo='word2vec')
sistema.aggiungi_documenti(articoli, titoli_articoli)

# Test con diverse query
query_test = [
    "Ho un gatto e voglio consigli",
    "Cerco modi per rilassarmi",
    "Voglio coltivare piante in casa",
    "Consigli per animali domestici"
]

print("\n🔍 TEST RACCOMANDAZIONI:\n")

for query in query_test:
    print(f"🎯 QUERY: '{query}'")
    raccomandazioni = sistema.raccomanda(query, n_raccomandazioni=3)
    
    print("📋 RACCOMANDAZIONI:")
    for racc in raccomandazioni:
        print(f"   {racc['rank']}. {racc['titolo']} (sim: {racc['similarita']:.3f})")
        print(f"      {racc['documento'][:60]}...")
    
    print("-" * 70)

# Visualizzazione delle raccomandazioni
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

for i, query in enumerate(query_test):
    raccomandazioni = sistema.raccomanda(query, n_raccomandazioni=5)
    
    titoli_racc = [r['titolo'] for r in raccomandazioni]
    similarita_racc = [r['similarita'] for r in raccomandazioni]
    
    bars = axes[i].barh(range(len(titoli_racc)), similarita_racc, 
                       color=['gold', 'silver', 'orange', 'lightblue', 'lightgreen'][:len(titoli_racc)])
    
    axes[i].set_yticks(range(len(titoli_racc)))
    axes[i].set_yticklabels(titoli_racc)
    axes[i].set_xlabel('Similarità')
    axes[i].set_title(f'Query: "{query[:20]}..."')
    
    # Aggiungi valori
    for bar, sim in zip(bars, similarita_racc):
        axes[i].text(bar.get_width() + 0.01, bar.get_y() + bar.get_height()/2,
                    f'{sim:.2f}', va='center', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n💡 VANTAGGI SISTEMA CON EMBEDDINGS:")
print("✅ Cattura similarità semantica")
print("✅ Non richiede parole esatte")
print("✅ Gestisce sinonimi e concetti correlati")
print("✅ Scalabile a grandi dataset")
print("✅ Personalizzabile per domini specifici")

print("\n🚀 POSSIBILI MIGLIORAMENTI:")
print("🔧 Embeddings pre-addestrati più grandi")
print("🔧 Combinazione di più metodi")
print("🔧 Feedback degli utenti")
print("🔧 Filtri per categoria/data")
print("🔧 Embeddings contestuali (BERT, etc.)")

## 🎯 Esercizio Pratico: Il Tuo Sistema di Embeddings

Ora tocca a te! Crea un sistema personalizzato:

In [ ]:
# ESERCIZIO: Crea il tuo sistema di embeddings
print("🎯 ESERCIZIO: Costruisci il Tuo Sistema di Embeddings\n")

class MioSistemaEmbeddings:
    def __init__(self, metodo='combinato'):
        self.metodo = metodo
        self.vocabolario = {}
        self.embeddings_parole = {}
        
    def addestra(self, documenti):
        """TODO: Implementa il tuo metodo di addestramento"""
        print(f"🏋️ Addestrando con metodo: {self.metodo}")
        
        # TODO: Implementa qui la tua logica
        # Suggerimenti:
        # 1. Combina TF-IDF + Word2Vec
        # 2. Usa pesi diversi per parole diverse
        # 3. Aggiungi informazioni di posizione
        # 4. Considera la lunghezza del documento
        
        # Esempio base:
        if self.metodo == 'combinato':
            # Combina TF-IDF e Word2Vec
            pass
        
        print("✅ Addestramento completato!")
    
    def documento_to_embedding(self, documento):
        """TODO: Converte documento in embedding"""
        # TODO: Implementa la tua logica di conversione
        
        # Esempio semplice: usa Word2Vec
        return documento_to_w2v(documento, model_w2v)
    
    def trova_simili(self, query, documenti, n=3):
        """TODO: Trova documenti simili"""
        query_emb = self.documento_to_embedding(query)
        
        similarita = []
        for doc in documenti:
            doc_emb = self.documento_to_embedding(doc)
            sim = cosine_similarity([query_emb], [doc_emb])[0][0]
            similarita.append(sim)
        
        # Ordina e restituisci top N
        indici = np.argsort(similarita)[::-1][:n]
        risultati = [(documenti[i], similarita[i]) for i in indici]
        
        return risultati

# Test del tuo sistema
print("🧪 TEST DEL TUO SISTEMA:\n")

# Dati di test
docs_test = [
    "Il machine learning è una branca dell'intelligenza artificiale",
    "I gatti sono animali domestici molto indipendenti",
    "La programmazione Python è utile per data science",
    "Gli algoritmi di deep learning usano reti neurali",
    "I cani sono compagni fedeli e giocherelloni"
]

mio_sistema = MioSistemaEmbeddings(metodo='combinato')
mio_sistema.addestra(docs_test)

# Test query
query_test = "Voglio imparare l'intelligenza artificiale"
print(f"🔍 Query: '{query_test}'")

risultati = mio_sistema.trova_simili(query_test, docs_test, n=3)

print("\n📋 Risultati:")
for i, (doc, sim) in enumerate(risultati, 1):
    print(f"   {i}. (sim: {sim:.3f}) {doc}")

print("\n💪 SFIDE PER TE:")
sfide = [
    "🔧 Implementa un metodo che combina TF-IDF e Word2Vec",
    "⚖️ Aggiungi pesi diversi per parole importanti",
    "📍 Considera la posizione delle parole nel documento",
    "📏 Normalizza per la lunghezza del documento",
    "🎯 Aggiungi filtri per categoria o data",
    "📊 Implementa metriche di valutazione",
    "🔄 Crea un sistema di feedback per migliorare",
    "🌐 Gestisci documenti multilingue"
]

for sfida in sfide:
    print(f"   {sfida}")

print("\n🎓 SUGGERIMENTI:")
suggerimenti = [
    "📚 Studia embeddings pre-addestrati (GloVe, FastText)",
    "🔬 Sperimenta con diversi metodi di aggregazione",
    "📈 Misura sempre le prestazioni su dati reali",
    "🔍 Analizza i casi in cui il sistema fallisce",
    "💡 Considera il contesto del tuo dominio applicativo"
]

for suggerimento in suggerimenti:
    print(f"   {suggerimento}")

# Spazio per il tuo codice
print("\n" + "="*70)
print("✏️ SPAZIO PER IL TUO CODICE:")
print("   Modifica la classe MioSistemaEmbeddings sopra!")
print("   Testa con i tuoi dati!")
print("   Confronta con i metodi standard!")
print("="*70)

## 🎓 Cosa Abbiamo Imparato

Congratulazioni! Hai completato il notebook sui Word Embeddings:

In [ ]:
# Riassunto finale
print("🎉 CONGRATULAZIONI! Hai completato il Notebook 3\n")

concetti_appresi = [
    "✅ Perché l'AI ha bisogno di rappresentazioni numeriche",
    "✅ Implementazione Bag of Words da zero",
    "✅ Calcolo TF-IDF e sua interpretazione",
    "✅ Concetti di Word2Vec e embeddings densi",
    "✅ Visualizzazione embeddings con PCA e t-SNE",
    "✅ Confronto sistematico dei metodi",
    "✅ Applicazione pratica: sistema di raccomandazione",
    "✅ Metriche di similarità (coseno)",
    "✅ Analisi di cluster semantici"
]

print("📚 CONCETTI APPRESI:")
for concetto in concetti_appresi:
    print(f"   {concetto}")

print("\n🛠️ COMPETENZE PRATICHE:")
competenze = [
    "🔢 Implementazione algoritmi di embedding",
    "📊 Visualizzazione spazi vettoriali",
    "🔍 Calcolo similarità semantica",
    "⚖️ Confronto metodi diversi",
    "🎯 Costruzione sistemi di raccomandazione",
    "📈 Valutazione qualità embeddings"
]

for competenza in competenze:
    print(f"   {competenza}")

# Tabella riassuntiva metodi
print("\n📊 TABELLA RIASSUNTIVA METODI:")
df_riassunto = pd.DataFrame({
    'Metodo': ['Bag of Words', 'TF-IDF', 'Word2Vec'],
    'Tipo': ['Sparso', 'Sparso', 'Denso'],
    'Dimensionalità': ['Alta (= vocabolario)', 'Alta (= vocabolario)', 'Bassa (50-300)'],
    'Semantica': ['No', 'Parziale', 'Sì'],
    'Velocità': ['Veloce', 'Veloce', 'Media'],
    'Uso Principale': ['Classificazione base', 'Ricerca documenti', 'AI avanzata']
})

print(df_riassunto.to_string(index=False))

print("\n🚀 PROSSIMI PASSI:")
print("   📖 Notebook 4: Reti Neurali Ricorrenti")
print("   📖 Notebook 5: Transformer e LLM")
print("   📖 Notebook 6: Vector Stores e RAG")

print("\n💡 SUGGERIMENTI PER CONTINUARE:")
print("   • Sperimenta con embeddings pre-addestrati")
print("   • Testa su dataset del tuo dominio")
print("   • Confronta con embeddings contestuali (BERT)")
print("   • Costruisci applicazioni pratiche")

print("\n🌟 RICORDA:")
print("   Gli embeddings sono la base di tutta l'AI moderna.")
print("   La qualità degli embeddings determina il successo del modello!")

# Badge di completamento
fig, ax = plt.subplots(figsize=(8, 6))

circle = plt.Circle((0.5, 0.5), 0.4, color='lightgreen', alpha=0.8)
ax.add_patch(circle)

ax.text(0.5, 0.6, '🔢', ha='center', va='center', fontsize=40)
ax.text(0.5, 0.45, 'COMPLETATO', ha='center', va='center', 
        fontsize=14, fontweight='bold')
ax.text(0.5, 0.35, 'Notebook 3', ha='center', va='center', 
        fontsize=12)
ax.text(0.5, 0.25, 'Word Embeddings', ha='center', va='center', 
        fontsize=10, style='italic')

ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.axis('off')
ax.set_title('Badge di Completamento', fontsize=16, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n🔬 APPROFONDIMENTI AVANZATI (opzionali):")
print("1. Embeddings contestuali (ELMo, BERT)")
print("2. FastText per parole fuori vocabolario")
print("3. Sentence embeddings (Doc2Vec, Universal Sentence Encoder)")
print("4. Embeddings multimodali (testo + immagini)")
print("5. Fine-tuning embeddings per domini specifici")

print("\n🎯 PROGETTI PRATICI SUGGERITI:")
print("• Sistema di ricerca semantica")
print("• Classificatore di sentimenti")
print("• Chatbot con comprensione semantica")
print("• Analizzatore di similarità documenti")
print("• Sistema di tag automatici")

## 📝 Note e Riflessioni

Usa questa sezione per annotare le tue riflessioni:

**Le mie note sui Word Embeddings:**

- 
- 
- 

**Domande per approfondire:**

- 
- 
- 

**Idee per applicazioni:**

- 
- 
- 

**Esperimenti da provare:**

- 
- 
- 